# Итераторы

Если просматривается набор данных, не помещающийся целиком в память, то нужен способ выполнять ее отложенно, т.е. по одному элементу и по запросу. Именно в этом смысл паттерна **Итератор**.

**Итератором** является любой объект, в котором реализован метод `__next__()`, т.е. объект, с которым может работать функция `next()`.

Объект явяется **итерируемым** если выполняется хотя бы одно из условий:
- он является физически последовательностью, и потому в нем реализован метод `__getitem__()` (который принимает индексы, начинающиеся с нуля) на основе которого из объекта можно сделать итератор;
- в нем реализован метод `__iter__()`, который возвращает итератор;
- в нем реализован метод `__next__()`, т.е. объект уже является итератором.

Для того, чтобы выяснить, является ли объект *итератором*, можно импортировать класс `Iterator` и проверить, является ли объект экземпляром этого класса:

In [ ]:
from collections.abc import Iterator
L = [1, 2, 3]
isinstance(L, Iterator)

False

> Проверить, является ли объект `x` *итерируемым*, можно вызвав функцию `iter()`, и перехватив исключение `TypeError`, если оно возникает. Это надежнее, чем использование `isinstance(x, Iterable)` (как то делалось выше для проверки, является ли объект итератором), так как в этом случае фактически проверяется реализован ли в объекте метод `__iter__()`. В общем случае, объект может быть итерируемым и без метода `__iter__()`.

### `iter()`

Из итерируемого объекта всегда можно получить итератор при помощи встроенной функции `iter()`. Функция выполняет следующие действия:
1. Проверяется, реализует ли объект метод `__iter__()`, и, если да, вызывает его, чтобы получить итератор.
1. Если метод `__iter__()` не реализован, но реализован метод `__getitem__()`, то Python создает итератор, который пытается извлекать элементы по порядку, начиная с индекса `0`.
1. Если и это не получается, то возбуждается исключение – обычно с сообщением `object is not iterable`.

Создадим итератор из списка. Так как в классе `list` определен метод `__iter__()`, функция `iter()` вызовет этот метод и вернет итератор (объект класса `list_iterator`), возращаемый методом.

In [ ]:
L = [i for i in range(10)]

L_iterator = iter(L)
type(L_iterator)

list_iterator

In [ ]:
L.__iter__()    # непосредственный вызов метода

При использовании цикла `for`, в котором перебираются элементы последовательности, за кулисами создается итератор, т.е. последовательность оборачивается в функцию `iter()`, а перебор происходит вызовом функции `next()`.

Применение функции к итератору, возвращается сам итератор, так как в итераторах метод `__iter__()` возвращает просто `self` (таким образом итератор является итерируемым объетом). Поэтому при использовании итератора в цикле `for` оборачивание итератора в функцию `iter()` ничего не меняет:

In [ ]:
iter(L_iterator) is L_iterator

True

In [ ]:
for item in L_iterator:
    print(item, end=' ')

0 1 2 3 4 5 6 7 8 9 

### `next()`
Функция `next()` вызывает метод `__next__()` итератора, который возвращает следующий элемент. Если в функцию будет передан не итератор, возникнет исключение `TypeError`. Если элементы итератора исчерпались, возбуждается исключение `StopIteration`.

In [ ]:
L_iterator = iter([i for i in range(10)])

In [ ]:
try:
    while True:
        item = next(L_iterator)
        print(item, end=" ")
except StopIteration:
    print('\nend')


0 1 2 3 4 5 6 7 8 9 
end


### Итератор из словаря

Если в качестве аргумента передать в `iter()` словарь, то будет возвращен итератор, содержащий ключи словаря:

In [ ]:
d = {'a': 12, 'b': 13, 'c': 14}
iDict = iter(d)
print(f'{ next(iDict) = }')

 next(iDict) = 'a'


Для того, чтобы итератор содержал пары (ключ, значение), необходимо передать в функцию `iter()` не сам словарь, а объект класса `dict_items`, возвращаемый методом `items()`:

In [ ]:
d = {'a': 12, 'b': 13, 'c': 14}
iDict = iter(d.items())
print(f'{ next(iDict) = }')

 next(iDict) = ('a', 12)


### Сброс итератора

Поскольку от итератора требуются только методы `__next__()` и `__iter__()`, не существует другого способа узнать, остались ли еще элементы, как только вызвать `next()` и перехватить исключение `StopInteration`.

Сбросить итератор тоже невозможно. Чтобы начать обход сначала, нужно вызвать функцию `iter()` для *итерируемого* объекта и получить от нее новый итератор. Вызов `iter()` для самого итератора не поможет, поскольку, метод `__iter__()` итератора возвращает `self`, так что таким способом исчерпанный итератор не восстановить.

### Конвертация в `list`

При конвертации в список итератор опустошается, а в список попадают оставшиеся элементы итератора.

In [ ]:
c = iter(['a', 'b', 'c', 'd'])  # итератор
d = list(c)
e = list(c)
print(f"{d = }, {e = }")


d = ['a', 'b', 'c', 'd'], e = []


### Определение метода `__iter__()` при создании класса

Итерируемый объект никогда не должен выступать в роли итератора для себя самого. Иными словами, итерируемые объекты должны реализовывать метод `__iter__()`, но не `__next__()`. С другой стороны, итераторы для удобства должны быть итерируемыми объектами. Просто метод `__iter__()` должен возвращать `self`. Такой подход позволяет создавать на основе одного итерируемого объекта несколько независимых итераторов для нескольких активных обходов. Создание классов, объекты которых являются одновременно итерируемыми и итераторами над самими собой является типичным антипаттероном.

Пример создания итерируемого класса с методом `__iter__()` в книге Рамальо на страницах 443, 447 и 448.

### `zip()`
Функция `zip()` (замок, молния) позволяет из двух и более последовательностей создать итератор состоящий из кортежей, содержащих соответствующие по индексам элементы последовательностей.

> Вместо последовательностей в общем случае можно использовать множества и словари. При использовании словарей элементами, которые попадут в кортежи, будут ключи. Следует однако иметь в виду, что при использовании множеств и словарей последовательность элементов может оказаться непредсказуемой.

In [ ]:
s = 'abcd'
t = (0, 1, 2, 3, 4, 5)
m = {12, 32, 34, 45, 56}
# m = {'x': 10, 'y': 30, 'z': 40, 'w': 50}

a = zip(s, t, m)    # создается итератор
type(a)

zip

In [ ]:
next(a)      # извлечение очередного элемента

('a', 0, 32)

In [ ]:
for i, j, k in a:      # извлечение оставшихся элементов в цикле
    print(i, j, k)

> В python 3.10 функция `zip` может принимать необязательный логический параметр `strict`, который позволяет проверять равенство длин последовательностей, если `strict=True`, то возникнет ошибка `ValueError`, в случае, если одна последовательность короче другой

С использованием функции `zip()` можно создать из двух последовательностей словарь, или добавить новые значения в уже существующий:

In [ ]:
s = 'abcd'
t = [0, 1, 2, 3, 4, 5]

dictionary = dict(zip(s, t))    # создается словарь
print(dictionary)

{'a': 0, 'b': 1, 'c': 2, 'd': 3}


In [ ]:
t = [i+1 for i in t]
dictionary.update(zip(s, t))    # словарь обновляется
print(dictionary)

{'a': 1, 'b': 2, 'c': 3, 'd': 4}


Наряду с тем, что можно разбить $n$ последовательностей длины $m$ на кортежи по $n$ элементов, функция `zip()` позволяет произвести обратную операцию: из полседовательности $m$ котрежей, содержащих по $n$ элементов, получить $n$ последовательностей длины $m$. Произведем как прямую, так и обратную операции:

In [7]:
m = 10
a = tuple(range(m))
b = tuple(range(m, 2*m))
c = tuple(range(2*m, 3*m))
t = tuple(zip(a, b, c))
t

((0, 10, 20),
 (1, 11, 21),
 (2, 12, 22),
 (3, 13, 23),
 (4, 14, 24),
 (5, 15, 25),
 (6, 16, 26),
 (7, 17, 27),
 (8, 18, 28),
 (9, 19, 29))

Фактически, мы просто распаковываем кортеж `t`, и передаем таким образом в `zip()` $m$ последовательностей: 

In [6]:
a, b, c = zip(*t)
a, b, c

((0, 1, 2, 3, 4, 5, 6, 7, 8, 9),
 (10, 11, 12, 13, 14, 15, 16, 17, 18, 19),
 (20, 21, 22, 23, 24, 25, 26, 27, 28, 29))

### `enumerate()`

Функция `enumerate()` создает из итерируемого объекта итератор, который содержит кортежи пар (индекс, значение), а для словаря (индекс, ключ).

In [ ]:
b = ['a', 'b', 'c', 'd']
for i, v in enumerate(b):
    print(i, v)

0 a
1 b
2 c
3 d


In [ ]:
b = {'x': 23, 'y': 24, 'z': 25}
for index, key in enumerate(b):
    value = b[key]
    print(f" = {index = }, {key = }, {value = }")

 = index = 0, key = 'x', value = 23
 = index = 1, key = 'y', value = 24
 = index = 2, key = 'z', value = 25


# Генераторы-выражения (genexp)

**Генераторы-выражения** (*generator expressions*) синтаксически похожи на генераторы списков, но с круглыми скобками `()`, а не с квадратными. Выражения возвращают объекты класса `generator`, поведение которого очень похоже на поведение итераторов. Принципиальное отличие заключается в том, что в отличие от итератора, который уже содержит в себе все значения последовательности, генераторы вычисляют каждое очередное значение при вызове функции `next()`. Аналогично итераторам, при исчерпании последовательности генератор при вызове функции `next()` возбуждает исключение `StopIteration`.

In [ ]:
g = ((x, x**2) for x in range(5))
type(g)

generator

In [ ]:
next(g)
next(g)

(1, 1)

In [ ]:
for a, b in g:
    print(a, b)

2 4
3 9
4 16


# Генераторные функции (оператор `yield`)

В **генераторных функциях** использются операторы `yield`, чтобы возвращать по одному результату за раз при вызове функции `next()`, приостанавливая выполнение с сохранением состояния, и возобновлять выполнение при очередном вызове `next()`. Функция `next()` возвращает каждый раз значение выражения, следующего за оператором `yield`.

Генераторная функция строит объект-генератор, обертывающий тело функции, который приостанавливает выполнение функции на каждом операторе `yield`. Этот генератор также как и генератор, полуаемый из выражения-генератора. После того, как функция полностью выполнится, очередной вызов функции `next()` возбуждает исключение `StopIteration`.

Из-за того, что генераторная функция (как и выражения генераторы) не создает сразу весь результирующий список, экономится пространство памяти и время вычисления распределяется по запросам результатов за счет протокола итераций.

In [ ]:
def gen_countdoun(n):
    while n != 0:
        yield n - 1
        n -= 1

In [ ]:
g = gen_countdoun(3)
print(next(g))
print(next(g))
print(next(g))
# print(next(g))  # приведет к исключению StopIteration

2
1
0


Вызов генераторной функции в цикле:

In [ ]:
for i in gen_countdoun(15):
    print(i, end=' ')

14 13 12 11 10 9 8 7 6 5 4 3 2 1 0 

В общем случае у генератроной функции может быть несколько операторов `yield`. При передаче объекта-генератора функции `next()` выполнение продолжается до следующего появления `yield` в теле функции:

In [ ]:
def even_odd(n):
    while n > 1:
        yield n
        yield n**2
        yield n**3
        n -= 1

In [ ]:
for v in even_odd(8):
    print(v, end=' ')

8 64 512 7 49 343 6 36 216 5 25 125 4 16 64 3 9 27 2 4 8 